In [1]:
import findspark
findspark.init()
import pyspark
sc = pyspark.SparkContext()

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=PySparkShell, master=local[*]) created by <module> at /Users/neilchainani/anaconda/lib/python2.7/site-packages/IPython/utils/py3compat.py:217 

In [3]:
txtFile=sc.textFile('../data/covtype.csv')
#Convert it into RDD of lists 
rdd=(txtFile.map(lambda x:x.split())
    .map(lambda x: x[0].strip("'").split(","))
    .map(lambda x:[float(v) for v in x])
    .map(lambda x: (x[-1]-1,x[0:-1])))

In [4]:
import itertools
soil_list =[]
for k in range(40):
    string = 'Soil_Type_' + str(k+1)
    soil_list.append(string)
WA_list =[]
for k in range(4):
    string = 'WA_' + str(k+1)
    WA_list.append(string)
names = [['Elevation'], ['Aspect'], ['Slope'], ['HDHyrdo'], ['VDHydro'], ['HDRoadways'], \
         ['9amHills'],['NoonHills'], ['3pmHills'], ['HDFirePoints'], WA_list,\
         soil_list, ['Cover_Type']]
columns = list(itertools.chain(*names))

In [5]:
import random

In [6]:
#selecting random features
m = 5 #No of features
indices = range(len(columns)-1)
random.shuffle(indices)
sampled_c = [columns[j] for j in indices[:m]]

In [7]:
#Code for implementing in rdd
%time sampled_rdd = rdd.map(lambda x: (x[0],[x[1][j] for j in indices[:m]]))

CPU times: user 24 µs, sys: 13 µs, total: 37 µs
Wall time: 30 µs


In [8]:
#Imports - move later to top
import numpy as np

In [23]:
#total counts of the data
t_count = rdd.count()

In [87]:
##Implementing the ID3 and testing the Entropy
test_index = 0
%time entrop_score = entropy_score(sampled_rdd,test_index)
%time entrop_score = entrop_score.take(1)[0][1]

CPU times: user 19.9 ms, sys: 5.2 ms, total: 25.1 ms
Wall time: 40 ms
CPU times: user 8.84 ms, sys: 2.15 ms, total: 11 ms
Wall time: 8 s


In [174]:
# Defined entropy as 1- sum(percentages squared)
def entropy_score(sampled_rdd,index):
    c=sampled_rdd.count()
    ent = sampled_rdd.map(lambda x:x[1][index]).map(lambda x:(x,1.0)).reduceByKey(lambda a,b:a+b)\
            .map(lambda x:(x[1]/c)*(x[1]/c)).reduce(lambda a,b:a+b)
    cats=sampled_rdd.map(lambda x:x[1][index]).distinct().collect()
    return 1-ent,cats

In [179]:
def best_attr(sampled_rdd,m):
    best_a=0
    best_s=100
    categories=[]
    for i in range(m):
        s,c=entropy_score(sampled_rdd,i)
        print i,s,c
        if s<best_s:
            best_a=i
            best_s=s
            categories=c
    return best_a, categories

In [71]:
%time entropy_score(sampled_rdd,1)

CPU times: user 32.7 ms, sys: 10 ms, total: 42.7 ms
Wall time: 35.1 s


0.000361372999140519

In [29]:
%time new_rdd = sampled_rdd.map(lambda x:x[1][test_index]).map(lambda x:(x,1.0)).reduceByKey(lambda a,b:a+b)\
            .map(lambda x:(x[0],(-x[1]/t_count)*(np.log2(x[1]/t_count))))\
            .map(lambda x:(1,x[1])).reduceByKey(lambda a,b:a+b)

CPU times: user 14.6 ms, sys: 3.09 ms, total: 17.7 ms
Wall time: 31.9 ms


In [30]:
%time print get_all_scores(sampled_rdd,m).collect()

[(0, 0.16207143284308864), (3, 0.027547699424360673), (6, 0.0025077543955656055), (9, 0.0095463517593474476), (1, 0.14900598458760123), (4, 6.5981678852644112), (7, 11.375626917177499), (2, 0.71862713465838812), (5, 0.031695711359855931), (8, 7.2756316788852136)]
CPU times: user 159 ms, sys: 36 ms, total: 195 ms
Wall time: 1min 31s


In [88]:
def infoGain(sampled_rdd, attr=0, target_attr=0):
    subset_entropy = 0
    val_freq = sampled_rdd.map(lambda x:x[1][attr]).map(lambda x:(x,1.0)).reduceByKey(lambda a,b:a+b)\
            .map(lambda x:(x[0],x[1]/t_count))
    val_freq = val_freq.collectAsMap()
    classes = sampled_rdd.map(lambda x: x[1][attr]).distinct().collect()
    print 'numClasses:',len(classes)
    for cls in classes:
        sub_rdd=sampled_rdd.filter(lambda x: x[1][attr]==cls)
        try:
            if sub_rdd.isEmpty() == True:
                continue
        except:
            print "Error in calculation. Moving on"
            continue
        subs_entropy = entropy_score(sub_rdd,target_attr)
        entropy = subs_entropy.collect()
        entropy = entropy[0][1]
        subset_entropy += entropy*val_freq[cls]
    return entropy_score(sampled_rdd,target_attr).collect()[0][1] - subset_entropy

In [154]:
from collections import Counter
import numpy as np
def IG(l):
    length=len(l)
    c=Counter()
    for v in l:
        c[v] += 1.0/length
    return 1-sum(np.multiply(c.values(),c.values()))

In [155]:
def infoGain2(sampled_rdd,attr=0, target_attr=0):
    c=sampled_rdd.count()
    pair_rdd=sampled_rdd.map(lambda x: (x[1][attr],x[1][target_attr]))
    list_rdd=pair_rdd.groupByKey().mapValues(lambda x: tuple(x)).map(lambda x: (x[0],x[1], len(x[1])/float(c)))
    gains_rdd=list_rdd.map(lambda x: (x[0],IG(x[1]),x[2]))
    return gains_rdd.map(lambda x: x[1]*x[2]).reduce(lambda a,b:a+b)

In [148]:
%time infoGain2(sampled_rdd, attr=0, target_attr=2)

0.00887232404524
CPU times: user 62.9 ms, sys: 14.8 ms, total: 77.7 ms
Wall time: 56.4 s


-0.022489466281998743

In [41]:
%time infoGain(sampled_rdd,attr=0,target_attr=1)

{0.0: 0.9762380122957873, 1.0: 0.02376198770421265}
0.530139880469
0.544515664964
CPU times: user 97.2 ms, sys: 22.2 ms, total: 119 ms
Wall time: 44.1 s


-0.39550968037623996

In [162]:
def choose_attribute(sampled_rdd, attribute, target_attrs):
    best_gain = 100.0
    best_attr = None
    for target in target_attrs:
        if target != attribute:
            gain = infoGain2(sampled_rdd,attribute, target)
            print target,gain
            if gain <= best_gain:
                best_gain = gain
                best_attr = target
    return best_gain,best_attr

In [76]:
%time choose_attribute(sampled_rdd,range(m),0)

0 1.34301799802e-11
1 0.00887231683732
2 0.00887168096634
3 0.00886996078236
4 0.00887212864823
CPU times: user 177 ms, sys: 40.9 ms, total: 218 ms
Wall time: 2min 5s


(0.0088723168373202911, 1)

In [139]:
sample = sc.parallelize([       (0,[1,1,0,1]),
         (0,[0,0,1,1]),
         (0,[1,1,0,2]),
         (0,[1,1,0,1]),
         (0,[0,0,1,0]),
         (0,[1,0,2,1]),
         (0,[2,0,1,1]),
         (0,[1,2,1,1])])

In [164]:
choose_attribute(sample,2,range(4))

0 0.3125
1 0.1875
3 0.354166666667


(0.1875, 1)

In [183]:
# get best starting attribute

start_split,cats=best_attr(sample,4)
for cat in cats:
    recu

0 0.53125 [0, 1, 2]
1 0.59375 [0, 1, 2]
2 0.59375 [0, 1, 2]
3 0.40625 [0, 1, 2]


(3, [0, 1, 2])

In [185]:
##
#separate branches

# send subtrees to get attributes, recursively
# stop tree once you reach a point of 0 info gain
def recursive(sub_rdd,attr,target_attrs):
    best_attr, cats=choose_attribute(sub_rdd,attr,target_attrs)
    for cat in cats:
        target_attrs=#create a list of attributes you haven't checked yet
        b,c=recursive(sub_rdd.filter(lambda x:x[1][best_attr]==cat),best_attr,target_attrs)
        if b==0: # no new information
            return c



[(0, [0, 0, 1, 0])]
[(0, [1, 1, 0, 1]), (0, [0, 0, 1, 1]), (0, [1, 1, 0, 1]), (0, [1, 0, 2, 1]), (0, [2, 0, 1, 1]), (0, [1, 2, 1, 1])]
[(0, [1, 1, 0, 2])]


The first cut is calculated with best_attr. Subsequent cuts can be calucluated with choose_attribute.